In [2]:
import pandas as pd
import psycopg2
import os

conn = psycopg2.connect(
    host="localhost",
    database="Superstore",
    user="postgres",
    password="halil@08"
)
cur = conn.cursor()

files = {
    "superstore_product": "superstore_product.xlsx",
    "superstore_order": "superstore_order.xlsx",
    "superstore_customer": "superstore_customer.xlsx",
    "product_stock": "product_stock.xlsx"
}

def create_table_from_df(table_name, df):
    cols = []
    for col in df.columns:
        col_clean = (
            col.strip()
               .replace(" ", "_")
               .replace("-", "_")
               .replace("/", "_")
               .replace(".", "_")
               .lower()
        )
        cols.append(f"{col_clean} TEXT")

    columns_sql = ", ".join(cols)

    cur.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});")
    conn.commit()

def insert_dataframe(table_name, df):
    clean_cols = [
        c.strip()
         .replace(" ", "_")
         .replace("-", "_")
         .replace("/", "_")
         .replace(".", "_")
         .lower()
        for c in df.columns
    ]

    for _, row in df.iterrows():
        values = [str(v).replace("'", "''") for v in row.values]

        placeholders = ", ".join(["%s"] * len(values))
        insert_sql = f"""
            INSERT INTO {table_name} ({', '.join(clean_cols)})
            VALUES ({placeholders});
        """
        cur.execute(insert_sql, values)
    conn.commit()

for table, filename in files.items():
    print(f"\nProcessing: {filename} → {table}")

    df = pd.read_excel(filename)
    create_table_from_df(table, df)
    insert_dataframe(table, df)

print("\nImported into PostgreSQL.")

cur.close()
conn.close()


Processing: superstore_product.xlsx → superstore_product

Processing: superstore_order.xlsx → superstore_order

Processing: superstore_customer.xlsx → superstore_customer

Processing: product_stock.xlsx → product_stock

Imported into PostgreSQL.
